In [137]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import numpy as np
from sklearn.tree import DecisionTreeRegressor,export_graphviz
from sklearn import tree
%matplotlib inline

In [138]:
train = pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\train.csv")
test =  pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\test.csv")

In [139]:
# store target as Y
train=train[train.y<=170]
Y_train = train["y"]
#train.drop(["y"], axis=1, inplace=True)
id_test = test["ID"]
# Concat both sets
num_train = len(train)
all_data = pd.concat([train, test])

In [140]:
def replaceNonNumtoNum(c,train,all_data):
    g_NonNum=train.groupby(c)['y'].agg(['mean']).reset_index()
    g_NonNum=g_NonNum.sort_values(by=('mean'),ascending=False)
    g_NonNum['rnk']=g_NonNum['mean'].rank(ascending=False)
    all_data=pd.merge(all_data, g_NonNum, how='left', on=c)
    all_data[c]=all_data.rnk
    all_data.drop(['mean','rnk'], axis=1, inplace=True)
    return all_data

In [141]:
for f in all_data.columns:
    if all_data[f].dtype=='object':
        all_data=replaceNonNumtoNum(f,train,all_data)

In [142]:
all_data.drop(["y"], axis=1, inplace=True)

In [143]:
all_data.drop(["ID"], axis=1, inplace=True)
#all_data.fillna(all_data.mean().astype(int))
#split both sets -- old habit ;)
X_train = all_data[:num_train]
X_test = all_data[num_train:]

In [144]:
all_data["X0"]=all_data["X0"].fillna(all_data["X0"].median())
all_data["X2"]=all_data["X2"].fillna(all_data["X2"].median())
all_data["X5"]=all_data["X5"].fillna(all_data["X5"].median())

In [145]:
X_train, X_cv, y_train, y_cv = train_test_split( X_train, Y_train, test_size = 0.2, random_state = 100)

In [104]:
d = []
# Decision tree tuning
for crtr in ['mse']:
    for md in [2,3,4,5,6,7,8,9,10,15,20,25]:
        for spltr in ['best','random']:
            for mss in [2,3,5,10,15,25,40,65,105]:
                for msl in [2,3,5,10,15,25,40,65,105]:
                    dts = DecisionTreeRegressor(criterion=crtr, max_depth=md,
                                max_features=None, max_leaf_nodes=None, min_samples_leaf=msl,
                                min_samples_split=mss, min_weight_fraction_leaf=0.0,
                                presort=False, random_state=100, splitter=spltr)
                    dts.fit(X_train, y_train)
                    y_pred = dts.predict(X_cv)
                    sip=r2_score(y_cv,y_pred)
                    print("score for {} criterion, {} max_depth, {} splitter, {} min_samples_split, {} min_samples_leaf is {}".format(crtr,md,spltr,mss,msl,sip))
                    d.append({'criterion': crtr, 'max_depth': md, 'splitter':spltr , 'min_samples_split' :mss, 'min_samples_leaf' : msl, 'score' : sip})

score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 2 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 3 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 5 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 10 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 15 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 25 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 40 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 65 min_samples_leaf is 0.5700562638175064
score for mse criterion, 2 max_depth, best splitter, 2 min_samples_split, 1

In [105]:
d=pd.DataFrame(d)
d.to_csv('bestmodelWithCategory2.csv', index=False)

In [163]:
dtr_tuned = DecisionTreeRegressor(criterion='mse', max_depth=6,
                                max_features=None, max_leaf_nodes=None, min_samples_leaf=105,
                                min_samples_split=105, min_weight_fraction_leaf=0.0,
                                presort=False, random_state=100, splitter='best')

In [164]:
dtr_tuned.fit(X_train, y_train)
r2_score(y_cv,dtr_tuned.predict(X_cv))

0.57123593224329661

In [165]:
# This statement builds a dot file.
cols = list(X_train.columns.values)
tree.export_graphviz(dtr_tuned, out_file='tree5d105.dot',feature_names  = cols) 

In [166]:
y_test = dtr_tuned.predict(X_test)
submission = pd.DataFrame({
        "ID": id_test,
        "y": y_test
    })
submission.to_csv('tree5d105.csv', index=False)

In [ ]:
X_test

In [125]:
X_test["X0"].describe()

count    4203.000000
mean       24.752082
std        11.752198
min         2.000000
25%        11.000000
50%        27.000000
75%        36.000000
max        47.000000
Name: X0, dtype: float64

In [ ]:
all_data["Fare"]=all_data["Fare"].fillna(train["Fare"].mean())